# Generation of images for marketing campaigns with Amazon Nova models

In this notebook we will demonstrate how to use the [Amazon Nova](https://aws.amazon.com/ai/generative-ai/nova/) models for the creation of images for marketing campaigns.

To execute the cells in this notebook you need to enable access to the following models on Bedrock:

* Amazon Nova Pro
* Amazon Nova Canvas

see [Add or remove access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html) to manage the access to models in Amazon Bedrock.

In this notebook we will explore how to generate images to accompany your ads using generative AI. For this demo we will be generating images for **advertising sporting products (balls, baseball bats, swimsuits, etc)**

In [ ]:
!pip install -U langchain-aws langchain-core langchain

In [ ]:
import boto3
import random
import os
import json
import io
import time
import datetime
import base64
import langchain_core

from PIL import Image
from IPython.display import Image as DisplayImage

from langchain_aws import ChatBedrock

from prompts.create_image_prompt_selector import get_meta_prompt_prompt_selector
from prompts.create_ad_concept_prompt_selector import get_ad_concept_prompt_selector
from structured_output.meta_prompt import MetaPrompt
from structured_output.ad_concept import AdConcept

from botocore.exceptions import ClientError
from botocore.config import Config

In [ ]:
langchain_core.globals.set_debug(True) # Set to True for enabling debugging stack traces

In [ ]:
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=Config(retries={'max_attempts': 20})
)

## Generate the campaign's concept with Amazon Nova Pro

As a first step we will generate a concept for our marketing campaign. We will use **Amazon Nova Pro** model to help us generate the campaign's concept based on the following input data:

* Sports category
* Sports subcategory
* Gender
* Age

therefore we will be generating campaigns based on an audience profile and the product we want to advertise

The choice of the Nova Pro model is for the reason that we need a model to perform a multi-step reasoning task using chain-of-thought and we also require it to be able to use tools (for structuring the outputs)

### Categories and audience

In [ ]:
categories = {
    "sports":["soccer", "basketball", "swimming", "boxing", "Weightlifting"],
    "electronics": ["television sets", "gaming consoles", "audio consoles", "smart home"],
    "furniture": ["bedrooms", "dinning sets", "sofas and armchairs", "tables and chairs"],
    "toys": ["action figures", "board games", "building sets", "collectibles", "card games", "puzzles"],
    "clothing": ["swimswear", "pajamas", "jackets", "shirts", "pants"]
}

category_keys = list(categories.keys())

audience_gender = ["male", "female"]
age_buckets = [(0,4), (5,9), (10,15), (15, 22), (22, 30), (30, 40), (40, 50), (50, 65), (65, 75), (75, 100)]

In [ ]:
category = category_keys[random.randint(0, len(category_keys)-1)]
subcategories = categories[category]
subcategory = subcategories[random.randint(0, len(subcategories)-1)]
gender = audience_gender[random.randint(0,1)]
age_range = age_buckets[random.randint(0, len(age_buckets)-1)]

print(f"Category: {category}")
print(f"Subcategory: {subcategory}")
print(f"Gender: {gender}")
print(f"Age: between {age_range[0]} and {age_range[1]} years old")

## Create the concept for the campaign using LLMs

First, given the audience profile and the product to advertise we will generate the general concept of the marketing campaign using the Nova Micro model. We ask the model to generate the following:

* reasoning: The model's reasoning for proposing that campaigns concept
* concept: The overall campaign's concept
* visual_concept: The visual concept of the campaign
* image: The description of the image for the campaign based on the generated visual concept

For this task we make use of [prompt templates](https://python.langchain.com/docs/concepts/prompt_templates/) and [structured_output](https://python.langchain.com/docs/concepts/structured_outputs/) in langchain

In [ ]:
CAMPAIGN_CONCEPT_GENERATION_MODEL_PARAMETERS = {
    "max_tokens": 3000,
    "temperature": 0.7,
    "top_k": 40,
}

NOVA_MICRO_MODEL_ID = "us.amazon.nova-pro-v1:0" # Cross Region Inference profile

In [ ]:
bedrock_llm_nova = ChatBedrock(
    model_id=NOVA_MICRO_MODEL_ID,
    model_kwargs=CAMPAIGN_CONCEPT_GENERATION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # Langchain object to interact with NOVA models through Bedrock

In [ ]:
AD_CONCEPT_PROMPT_SELECTOR = get_ad_concept_prompt_selector("en") # Amazon Nova Canvas officially only supports english prompts

In [ ]:
nova_ad_concept_prompt_template = AD_CONCEPT_PROMPT_SELECTOR.get_prompt(NOVA_MICRO_MODEL_ID)
structured_nova_ad_concept = bedrock_llm_nova.with_structured_output(AdConcept)

In [ ]:
nova_ad_concept_prompt_template.format(
    products=f"Category: {category}\nSubcategory: {subcategory}",
    audience=f"Gender: {gender}\nAge range: between {age_range[0]} and {age_range[1]}"
)

In [ ]:
langchain_ad_concept_llm = nova_ad_concept_prompt_template | structured_nova_ad_concept

In [ ]:
ad_concept = langchain_ad_concept_llm.invoke(
    {
        "products": f"Category: {category}\nSubcategory: {subcategory}",
        "audience": f"Gender: {gender}\nAge range: between {age_range[0]} and {age_range[1]}"
    }
)

In [ ]:
ad_concept

In [ ]:
ad_concept.campaign_concept

In [ ]:
ad_concept.visual_concept

In [ ]:
ad_concept.image_description

## Meta-prompting

Text-to-image foundation models require well crafted prompts to generate accurate images. We will leverage Amazon Nova Pro to create prompts for our Amazon Canvas model, this is a technique called [meta-prompting](https://www.prompthub.us/blog/a-complete-guide-to-meta-prompting).

In [ ]:
IMAGE_PROMPT_GENERATION_MODEL_PARAMETERS = {
    "max_tokens": 1500,
    "temperature": 0.7,
    "top_k": 40,
}

NOVA_MICRO_MODEL_ID = "us.amazon.nova-pro-v1:0" # Cross Region Inference profile

In [ ]:
bedrock_llm_nova_prompt_gen = ChatBedrock(
    model_id=NOVA_MICRO_MODEL_ID,
    model_kwargs=IMAGE_PROMPT_GENERATION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # Langchain object to interact with NOVA models through Bedrock

In [ ]:
META_PROMPT_PROMPT_SELECTOR = get_meta_prompt_prompt_selector("en") # Amazon Nova Canvas officially only supports english prompts

In [ ]:
nova_img_meta_prompt_prompt_template = META_PROMPT_PROMPT_SELECTOR.get_prompt(NOVA_MICRO_MODEL_ID)
structured_nova_meta_prompt = bedrock_llm_nova_prompt_gen.with_structured_output(MetaPrompt)

In [ ]:
nova_img_meta_prompt_prompt_template.format(text=ad_concept.image_description)  # Create meta-prompt from image description

In [ ]:
meta_prompt_llm = nova_img_meta_prompt_prompt_template | structured_nova_meta_prompt

In [ ]:
text_to_image_meta_prompt = meta_prompt_llm.invoke({"text": ad_concept.image_description})

In [ ]:
text_to_image_meta_prompt

In [ ]:
text_to_image_meta_prompt.prompt

## Image generation with Amazon Nova Canvas

We have the following data to create our campaign's images

In [ ]:
print(f"Category: {category}")
print(f"Subcategory: {subcategory}")
print(f"Gender: {gender}")
print(f"Age: between {age_range[0]} and {age_range[1]} years old")

For which Nova Pro generated the following **campaign concept**

In [ ]:
ad_concept.campaign_concept

**visual concept**

In [ ]:
ad_concept.visual_concept

and **image description**

In [ ]:
ad_concept.image_description

Then, from the image description and using meta-prompting we have created the following **image generation prompt for Nova Canvas**:

In [ ]:
text_to_image_meta_prompt.prompt

### Generate images using Amazon Nova Canvas

In [ ]:
random_seed = random.randint(0, 214783647)

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": text_to_image_meta_prompt.prompt,
            #"negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 720,        # Supported height list in the docs 
            "width": 1280,         # Supported width list in the docs
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": random_seed             # Range: 0 to 214783647
        }
    }
)

In [ ]:
# Make model request
start_time = time.time()
response = bedrock_runtime.invoke_model(
    body=body,
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)
time_canvas = time.time() - start_time

# Process the image
response_body = json.loads(response.get("body").read())

print(f"Generation latency: {time_canvas}")

### Save and visualize generated image

In [ ]:
# Decode + save

folder_id = datetime.datetime.now().microsecond

for i, img_b64 in zip(range(len(response_body["images"])), response_body["images"]):
    
    os.makedirs(f"./images/canvas/{folder_id}", exist_ok=True)
    
    img = Image.open(
        io.BytesIO(
            base64.decodebytes(
                bytes(img_b64, "utf-8")
            )
        )
    )
    img.save(f"./images/canvas/{folder_id}/image_{i}.png")

In [ ]:
DisplayImage(filename=f'./images/canvas/{folder_id}/image_0.png')

## Exploring other Nova Canvas features

For the final section of this notebook we will explore other Canvas features such as:

* Outpainting
* Inpainting
* Background removal
* Image conditioning

for this demos we will use previously generated images under the [./images/canvas](./images/canvas) folder

## Loading the image

In [ ]:
image_url = f'./images/canvas/443875/image_0.png'

In [ ]:
DisplayImage(filename=image_url)

Since all of the additional features we will explore require the image loaded as a Base64 string we will first load out image into memory

In [ ]:
import base64

with open(image_url, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode()

### Background removal

Automatically identifies multiple objects in the input image and removes the background. The output image has a transparent background.

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "BACKGROUND_REMOVAL",
        "backgroundRemovalParams": {
            "image": base64_image
        }
    }
)

# Make model request
start_time = time.time()
response = bedrock_runtime.invoke_model(
    body=body,
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)
time_canvas = time.time() - start_time

# Process the image
response_body = json.loads(response.get("body").read())

print(f"Backgtound removal latency: {time_canvas}")

In [ ]:
img = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(response_body["images"][0], "utf-8")
        )
    )
)

img.show()

### Inpainting

Uses an image and a segmentation mask as input (either from the user or estimated by the model) and reconstructs the region within the mask. Use inpainting to remove masked elements and replace them with background pixels.

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "INPAINTING",
        "inPaintingParams": {
            "image": base64_image,
            "maskPrompt": "elderly woman",
            "text": "elderly athletic man wearing box gloves",
            "negativeText": "too much light"
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 768,        # Supported height list in the docs 
            "width": 1280,         # Supported width list in the docs
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": random_seed             # Range: 0 to 214783647
        }
    }
)

# Make model request
start_time = time.time()
response = bedrock_runtime.invoke_model(
    body=body,
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)
time_canvas = time.time() - start_time

# Process the image
response_body = json.loads(response.get("body").read())

print(f"Inpainting latency: {time_canvas}")

In [ ]:
img = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(response_body["images"][0], "utf-8")
        )
    )
)

img.show()

### Outpainting

Uses an image and a segmentation mask as input (either from the user or estimated by the model) and generates new pixels that seamlessly extend the region. Use precise outpainting to preserve the pixels of the masked image when extending the image to the boundaries. Use default outpainting to extend the pixels of the masked image to the image boundaries based on segmentation settings.

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "OUTPAINTING",
        "outPaintingParams": {
            "image": base64_image,
            "maskPrompt": "elderly woman hitting boxsack",
            "text": "another person wearing box gloves fighting the woman in a box ring. show the face of the other person",
            "negativeText": "too much light",
            "outPaintingMode": "DEFAULT",
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 720,        # Supported height list in the docs 
            "width": 1280,         # Supported width list in the docs
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": random_seed             # Range: 0 to 214783647
        }
    }
)

# Make model request
start_time = time.time()
response = bedrock_runtime.invoke_model(
    body=body,
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json", 
    contentType="application/json"
)
time_canvas = time.time() - start_time

# Process the image
response_body = json.loads(response.get("body").read())

print(f"Outpainting latency: {time_canvas}")

In [ ]:
img = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(response_body["images"][0], "utf-8")
        )
    )
)

img.show()

### Image conditioning

Uses an input reference image to guide image generation. The model generates output image that aligns with the layout and the composition of the reference image, while still following the textual prompt.

In [ ]:
generation_text = """A dynamic image of a group of young female soccer players celebrating a goal on a vibrant green field. The players are 
diverse in appearance, showcasing unity and inclusivity. The background features an energetic crowd and a setting sun, adding to the excitement 
and passion of the game."""

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "conditionImage": base64_image,
            "controlMode": "SEGMENTATION", 
            "controlStrength": 0.5,
            "text": generation_text,
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 720,        # Supported height list in the docs 
            "width": 1280,         # Supported width list in the docs
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": random_seed             # Range: 0 to 214783647
        }
    }
)

# Make model request
start_time = time.time()
response = bedrock_runtime.invoke_model(
    body=body,
    modelId="amazon.nova-canvas-v1:0",
    accept="application/json",
    contentType="application/json"
)
time_canvas = time.time() - start_time

# Process the image
response_body = json.loads(response.get("body").read())

print(f"Image conditioning latency: {time_canvas}")

In [ ]:
img = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(response_body["images"][0], "utf-8")
        )
    )
)

img.show()